In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import split
from pyspark.sql.functions import col
import pandas as pd
import matplotlib.pyplot as plt
import upsetplot

#créer une session dans le master
spark = SparkSession.builder.master("spark://172.20.53.96:7077").appName("WDC-stats").getOrCreate()
#spark = SparkSession.builder.master("local").appName("WDC-stats").getOrCreate()
        
#fichiers de config qui permettent de se connecter au serveur de stockage s3 qui contient les fichiers de DataCommons
endpoint_url = 'https://s3.os-bird.glicid.fr/'
aws_access_key_id = 'bbd95ea3c1174caa88345404b84e458f'
aws_secret_access_key = 'eaf2a72ecf9845f583af7f3513c44f25'
hadoopConf = spark._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', aws_access_key_id)
hadoopConf.set('fs.s3a.secret.key', aws_secret_access_key)
hadoopConf.set('fs.s3a.endpoint', endpoint_url)
hadoopConf.set('fs.s3a.path.style.access', 'true')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

hadoopConf.set('spark.worker.cleanup.enabled', 'true')
hadoopConf.set('fs.s3a.committer.name', 'magic')

In [ ]:
avgValues = spark.read.option("header",True) \
  .csv("s3a://test-out/average/top10000/**")

avgValues = avgValues.withColumn("average", col("average").cast("float"))
avgValues.printSchema()
avgValues.show(truncate=150)


In [ ]:
avgValues = avgValues.sort(avgValues.average.desc())
avgValues.show(30, truncate=150)

In [ ]:
withouterrors = avgValues.filter(avgValues.average != -1)
print(withouterrors.count())


In [ ]:
valuesLimit10 = spark.read.option("header",True) \
  .csv("s3a://test-out/structuredness/limit10top10000/**")
valuesLimit10.show(truncate=150)

valuesLimit10 = valuesLimit10.withColumn("average", col("average").cast("float"))
valuesLimit10.printSchema()

In [ ]:
valuesLimit10 = valuesLimit10.sort(valuesLimit10.average.desc())
valuesLimit10.show(30, truncate=150)

In [ ]:
filtered_df_avg_withoutSchema = avgValues.filter(~avgValues["type"].isin(types_in_schema))
filtered_df_avg_withoutSchema.show(30, truncate=150)

In [ ]:
filtered_df_avg = avgValues.filter(avgValues["type"].isin(types_in_schema))
filtered_df_avg.show(30, truncate=150)

In [ ]:
filtered_df_Limit10_withoutSchema = valuesLimit10.filter(~valuesLimit10["type"].isin(types_in_schema))
filtered_df_Limit10_withoutSchema.show(30, truncate=150) 

In [ ]:
filtered_df_Limit10 = valuesLimit10.filter(valuesLimit10["type"].isin(types_in_schema))
filtered_df_Limit10.show(30, truncate=150)

In [ ]:
readall = spark.read.option("header",True) \
  .csv("s3a://test-out/wdcfix/**")

csall=readall.groupby("pset").agg(f.sum("count").alias('count')).sort(f.desc("count"))

csall.createOrReplaceTempView("CSET")
csall.show(truncate=150)

In [ ]:
from pyspark.sql.functions import col
def findISA(topn = None):
    if(topn is not None):
        param = f"limit {topn}"
    else:
        param = ""
    
    sets = csall.withColumn("pset", split(csall["pset"], " "))
    
    distinct_predicate = sets.selectExpr("explode(pset) as predicate", "count") \
                    .groupBy("predicate").agg(f.sum(f.col("count")).alias("count"))  \
                    .createOrReplaceTempView("newsets")
    distinct_predicate=spark.sql(f"select * from newsets where predicate like '%isa:%' and count > 1 order by count desc {param}")    
    return distinct_predicate

In [ ]:
findIsaTop10000 = findISA(10000)
findIsaTop10000.printSchema()
findIsaTop10000.show(truncate=150)

In [ ]:
from pyspark.sql.functions import desc

test=filtered_df_avg.join(findIsaTop10000,filtered_df_avg.type == findIsaTop10000.predicate ,"inner").drop("predicate").filter(filtered_df_avg.average != -1).sort(filtered_df_avg.average.desc())
test=test.sort(desc('count'))
test.printSchema()
test.show(truncate=150)


In [ ]:
ToFilterOut = [ "isa:<schema.org/ItemList>", "isa:<schema.org/BreadcrumbList>", "isa:<schema.org/HowToSection>", "isa:<schema.org/HowToStep>", "isa:<schema.org/OfferCatalog>" ]
test1 = test.filter(~test1["type"].isin(ToFilterOut))
test1=test1.sort(desc('count'))
test1.show(30, truncate=150)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert Spark dataframe to Pandas dataframe
pandas_df = test1.toPandas()

# Create scatter plot
plt.scatter(pandas_df['average'], pandas_df['count'])

# Add labels and title
plt.xlabel('Average')
plt.ylabel('Count')
plt.title('Scatter plot of Average and Count')

# Show the plot
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert spark dataframe to pandas dataframe
df_pd = test1.toPandas()

# Create bar plot
ax = df_pd.plot(kind='bar', x='type', y='average', color='blue')

# Set labels and title
ax.set_xlabel("Type")
ax.set_ylabel("Average")
ax.set_title("Average by Type")

# Show plot
plt.show()


In [ ]:
def upsetplotofclass():
    persons=spark.sql("select pset,count from CSET where pset like '%isa:<tripinvites.com/hi-in/places/lucknow/shaheed-smarak/ImageObject>%' ")
    persons = persons.withColumn("pset", split(persons["pset"], " "))
    persons.createOrReplaceTempView("persons")
    top_persons= spark.sql("""
    SELECT  *
    FROM persons
    limit 15
    """)
    pandas_persons = top_persons.toPandas()

    # Create a list of sets for each row in the DataFrame
    pset_list = [set(x) for x in pandas_persons['pset']]


    # Create the UpSetplot
    upset_data = upsetplot.from_memberships(pset_list, data=pandas_persons['count'])
    upsetplot.plot(upset_data, show_counts=True)
    
    plt.show()

In [ ]:
upsetplotofclass()